In [1]:
import numpy as np
import import_ipynb
# import DecisionTree
import pandas as pd
import random
from __future__ import division, print_function
import DecisionTree
import math

importing Jupyter notebook from DecisionTree.ipynb
{'RM <= 7.436999999999999': [{'LSTAT <= 15.0': [{'DIS <= 1.2437999999999998': [50.0,
                                                                               {'RM <= 6.797': [{'LSTAT <= 3.3949999999999996': [50.0,
                                                                                                                                 {'LSTAT <= 3.63': [32.4,
                                                                                                                                                    {'LSTAT <= 3.745': [50.0,
                                                                                                                                                                        {'LSTAT <= 4.52': [30.375,
                                                                                                                                                                                           {'B <= 114.685': [1

In [2]:
word_labels = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "label"]
df_train = pd.read_csv("../HousingData/housing_train.txt", delim_whitespace=True, names = word_labels, header=None) 
df_test = pd.read_csv("../HousingData/housing_test.txt", delim_whitespace=True, names = word_labels, header=None) 
df_train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,label
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
# def train_test_split(df, test_size):
    
#     if isinstance(test_size, float):
#         test_size = round(test_size * len(df))

#     index_list = df.index.tolist()
#     test_indexes = random.sample(population=index_list, k=test_size)

#     test_df = df.loc[test_indexes]
#     train_df = df.drop(test_indexes)
    
#     return train_df, test_df

In [4]:
# random.seed(3)
# train_df, test_df = train_test_split(df, 0.20)

In [5]:
X_train = df_train.iloc[: , :-1]
y_train = df_train.iloc[: , -1]

X_test = df_test.iloc[: , :-1]
y_test = df_test.iloc[: , -1]

# x = X_train
# y = y_train

In [6]:
class DecisionTree():
    def __init__(self, x, y, idxs = None, min_leaf=2):
        if idxs is None: idxs=np.arange(len(y))
        self.x,self.y,self.idxs,self.min_leaf = x,y,idxs,min_leaf
        self.n,self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        for i in range(self.c): self.find_better_split(i)
        if self.score == float('inf'): return
        x = self.split_col
        lhs = np.nonzero(x<=self.split)[0]
        rhs = np.nonzero(x>self.split)[0]
        self.lhs = DecisionTree(self.x, self.y, self.idxs[lhs])
        self.rhs = DecisionTree(self.x, self.y, self.idxs[rhs])

    def find_better_split(self, var_idx):
        x,y = self.x.values[self.idxs,var_idx], self.y[self.idxs]
        sort_idx = np.argsort(x)
        sort_y,sort_x = y[sort_idx], x[sort_idx]
        rhs_cnt,rhs_sum,rhs_sum2 = self.n, sort_y.sum(), (sort_y**2).sum()
        lhs_cnt,lhs_sum,lhs_sum2 = 0,0.,0.

        for i in range(0,self.n-self.min_leaf-1):
            xi,yi = sort_x[i],sort_y[i]
            lhs_cnt += 1; rhs_cnt -= 1
            lhs_sum += yi; rhs_sum -= yi
            lhs_sum2 += yi**2; rhs_sum2 -= yi**2
            if i<self.min_leaf or xi==sort_x[i+1]:
                continue

            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            curr_score = lhs_std*lhs_cnt + rhs_std*rhs_cnt
            if curr_score<self.score: 
                self.var_idx,self.score,self.split = var_idx,curr_score,xi

    @property
    def split_name(self): return self.x.columns[self.var_idx]
    
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]

    @property
    def is_leaf(self): return self.score == float('inf')
    
    def __repr__(self):
        s = f'n: {self.n}; val:{self.val}'
        if not self.is_leaf:
            s += f'; score:{self.score}; split:{self.split}; var:{self.split_name}'
        return s

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx]<=self.split else self.rhs
        return t.predict_row(xi)

In [7]:
x = np.arange(0,50)
x = pd.DataFrame({'x':x})

In [8]:
# just random uniform distributions in differnt range

y1 = np.random.uniform(10,15,10)
y2 = np.random.uniform(20,25,10)
y3 = np.random.uniform(0,5,10)
y4 = np.random.uniform(30,32,10)
y5 = np.random.uniform(13,17,10)

y = np.concatenate((y1,y2,y3,y4,y5))
y = y[:,None]
y

array([[1.15275420e+01],
       [1.22338396e+01],
       [1.01724972e+01],
       [1.14340424e+01],
       [1.07941227e+01],
       [1.21577395e+01],
       [1.41545950e+01],
       [1.46618604e+01],
       [1.06909455e+01],
       [1.41977921e+01],
       [2.13657803e+01],
       [2.13770323e+01],
       [2.36537924e+01],
       [2.29749762e+01],
       [2.19608594e+01],
       [2.01447194e+01],
       [2.19338378e+01],
       [2.41274588e+01],
       [2.02358898e+01],
       [2.39291739e+01],
       [2.33160194e+00],
       [4.09611903e-01],
       [6.56922882e-03],
       [3.30813567e+00],
       [1.94070729e+00],
       [5.85511349e-01],
       [6.49785291e-01],
       [1.03779464e+00],
       [1.43573934e+00],
       [2.66588322e+00],
       [3.01209563e+01],
       [3.07590509e+01],
       [3.17429623e+01],
       [3.19019267e+01],
       [3.17276233e+01],
       [3.12116446e+01],
       [3.08943954e+01],
       [3.03460898e+01],
       [3.16285325e+01],
       [3.09791621e+01],


In [9]:
# y = y[:, None]
# y

In [10]:
x.shape, y.shape

((50, 1), (50, 1))

In [11]:
def std_agg(cnt, s1, s2): 
    return math.sqrt((s2/cnt) - (s1/cnt)**2)

In [17]:
xi = x # initialization of input
yi = y # initialization of target
# x,y --> use where no need to change original y
ei = 0 # initialization of error
n = len(yi)  # number of rows
predf = 0 # initial prediction 0

for i in range(30): # like n_estimators
    tree = DecisionTree(xi,yi)
    print(tree)
    tree.find_better_split(0)
    
    r = np.where(xi == tree.split)[0][0]    
    
    left_idx = np.where(xi <= tree.split)[0]
    right_idx = np.where(xi > tree.split)[0]
    
    predi = np.zeros(n)
    np.put(predi, left_idx, np.repeat(np.mean(yi[left_idx]), r))  # replace left side mean y
    np.put(predi, right_idx, np.repeat(np.mean(yi[right_idx]), n-r))  # right side mean y
    
    predi = predi[:,None]  # make long vector (nx1) in compatible with y
    predf = predf + predi  # final prediction will be previous prediction value + new prediction of residual
    
    ei = y - predf  # needed originl y here as residual always from original y    
    yi = ei # update yi as residual to reloop
    
    
    # plotting after prediction
    xa = np.array(x.x) # column name of x is x 
    order = np.argsort(xa)
    xs = np.array(xa)[order]
    ys = np.array(predf)[order]
    
#     epreds = np.array(epred[:,None])[order]

#     f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize = (13,2.5))

#     ax1.plot(x,y, 'o')
#     ax1.plot(xs, ys, 'r')
#     ax1.set_title(f'Prediction (Iteration {i+1})')
#     ax1.set_xlabel('x')
#     ax1.set_ylabel('y / y_pred')

#     ax2.plot(x, ei, 'go')
#     ax2.set_title(f'Residuals vs. x (Iteration {i+1})')
#     ax2.set_xlabel('x')
#     ax2.set_ylabel('Residuals')

n: 50; val:16.403424466026582; score:418.6439098489; split:29; var:x


NameError: name 'epred' is not defined

In [13]:
xs

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [18]:
epreds

NameError: name 'epreds' is not defined